In [115]:
import pandas as pd
import numpy as np


# Data

## Descripcion

- ID                             : Número identificador del cliente
- CHECKING_BALANCE               : Saldo que posee el cliente en su cuenta corriente
- PAYMENT_TERM                   : Cantidad de días que el cvliente posee para pagar el préstamo
- CREDIT_HISTORY                 : Situación crediticia pasada del cliente
- LOAN_PURPOSE                   : Motivo del préstamo
- LOAN_AMOUNT                    : Monto del préstamo
- EXISTING_SAVINGS               : Saldo de cuenta de ahorros
- EMPLOYMENT_DURATION            : Cuántos años ha permanecido el cliente en su empleo
- INSTALLMENT_PERCENT            : Cantidad de cuotas en las que el préstamo debe ser pagado
- SEX                            : Sexo del cliente
- OTHERS_ON_LOAN                 : Denota la existencia de un garante u otro solicitante del préstamo
- CURRENT_RESIDENCE_DURATION     : Años que el cliente ha permanecido en su última residencia
- PROPERTY                       : Indica si el cliente posee alguna propiedad a su nombre
- AGE                            : Edad del cliente
- INSTALLMENT_PLANS              : Plan de financiamiento, que puede ser del banco, externo o ninguno
- HOUSING                        : Indica si el cliente posee una casa propia
- EXISTING_CREDITS_COUNT         : Número de préstamos que le han sido concedidos al cliente en el pasado
- JOB_TYPE                       : Tipo de empleo: 0 - desempleado, 1 - no calificado, 2 - autónomo, 3 - calificado
- DEPENDENTS                     : Número de personas con acceso a la cuenta
- TELEPHONE                      : Denota si el cliente tiene un número de teléfono registrado
- FOREIGN_WORKER                   Denot: si el cliente trabaja en un país fuera del banco

## Preprocesamiento

Datos directo de [Desafio 1 datos](https://github.com/maratonadev/desafio-1-2021/tree/main/assets/data)

In [116]:
answer = pd.read_csv("https://raw.githubusercontent.com/maratonadev/desafio-1-2021/main/assets/data/ANSWERS.csv")
acc = pd.read_csv("https://raw.githubusercontent.com/maratonadev/desafio-1-2021/main/assets/data/ACCOUNTS.csv")
dem = pd.read_csv("https://raw.githubusercontent.com/maratonadev/desafio-1-2021/main/assets/data/DEMOGRAPHICS.csv")
loa = pd.read_csv("https://raw.githubusercontent.com/maratonadev/desafio-1-2021/main/assets/data/LOANS.csv")

### Join

In [117]:
s1 = pd.merge(acc, dem, on = "ID", how = 'outer')
full = pd.merge(s1, loa, on = "ID", how = 'outer')

In [118]:
# full.head(3)

## Limpieza 

### Nas

Completar con

- Mediana: 
  - age, 
  - payment_term, 
  - installment_percent
- 0: 
  - checking_balance, 
  - existing_savings, 
  - existing_credits_count, 
  - {job_type}, 
  - dependents, 
  - telephone, 
  - foreing_worker, 
  - employment_duration, 
  - current_residence_duration, 
- string {ausencia}: 
  - credit_history -> NO_CREDITS
  - property -> UNKNOWN 
  - housing, -> FREE
  - installemnt_plans, -> NONE
  - others_on_loan -> NONE
- drop_na: 
  - sex, 
  - loan_amount
  - loan_purpose
  - allow

In [119]:
full['CHECKING_BALANCE'] = pd.to_numeric(full['CHECKING_BALANCE'], errors='coerce')
full['EXISTING_SAVINGS'] = pd.to_numeric(full['EXISTING_SAVINGS'], errors='coerce')
full['FOREIGN_WORKER'].fillna(0, inplace = True)

In [120]:
mediana = ['AGE', 'PAYMENT_TERM', 'INSTALLMENT_PERCENT']
full[mediana] = full[mediana].fillna(full[mediana].median())
full.fillna(
    value = {
        'CHECKING_BALANCE': 0
        , 'EXISTING_SAVINGS': 0
        , 'EXISTING_CREDITS_COUNT': 0
        , 'JOB_TYPE': 0  #####
        , 'DEPENDENTS': 0
        , 'TELEPHONE': 0
        , 'FOREING_WORKER': "0" 
        , 'EMPLOYMENT_DURATION': 0 
        , 'CURRENT_RESIDENCE_DURATION': 0 
        , 'CREDIT_HISTORY': 'NO_CREDITS'
        , 'PROPERTY': 'UNKNOWN'
        , 'HOUSING': 'FREE'
        , 'INSTALLMENT_PLANS': 'NONE'
        , 'OTHERS_ON_LOAN': 'NONE'
    }
    , inplace = True
)

In [121]:
full = full.dropna()
full
# full.isnull().sum()

,ID,CHECKING_BALANCE,CREDIT_HISTORY,EXISTING_SAVINGS,EXISTING_CREDITS_COUNT,SEX,AGE,JOB_TYPE,DEPENDENTS,TELEPHONE,FOREIGN_WORKER,EMPLOYMENT_DURATION,PROPERTY,HOUSING,CURRENT_RESIDENCE_DURATION,PAYMENT_TERM,INSTALLMENT_PLANS,INSTALLMENT_PERCENT,LOAN_PURPOSE,LOAN_AMOUNT,OTHERS_ON_LOAN,ALLOW
0,291655,-566.53,ALL_CREDITS_PAID_BACK,36.42,1.0,F,24.0,3.0,1.0,1.0,1.0,0.0,SAVINGS_INSURANCE,OWN,2.0,310.0,NONE,2.0,CAR_NEW,3503.0,NONE,1
1,657079,-1673.16,CREDITS_PAID_TO_DATE,22.90,1.0,F,30.0,3.0,1.0,0.0,1.0,5.0,CAR_OTHER,RENT,1.0,630.0,STORES,3.0,CAR_NEW,3729.0,NONE,1
2,256050,0.00,CREDITS_PAID_TO_DATE,450.35,2.0,M,40.0,3.0,1.0,0.0,1.0,4.0,UNKNOWN,OWN,4.0,868.0,NONE,3.0,CAR_USED,8516.0,NONE,0
3,1039151,-1471.03,PRIOR_PAYMENTS_DELAYED,805.71,2.0,M,28.0,1.0,1.0,0.0,1.0,0.0,CAR_OTHER,OWN,3.0,403.0,BANK,3.0,FURNITURE,2577.0,NONE,1
4,1002660,0.00,CREDITS_PAID_TO_DATE,295.03,1.0,M,19.0,2.0,1.0,0.0,1.0,3.0,REAL_ESTATE,OWN,3.0,810.0,NONE,4.0,BUSINESS,3750.0,NONE,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3931,582945,0.00,NO_CREDITS,0.00,0.0,M,38.0,3.0,1.0,0.0,1.0,9.0,SAVINGS_INSURANCE,OWN,4.0,713.0,NONE,4.0,FURNITURE,6959.0,NONE,0
3932,213735,0.00,NO_CREDITS,0.00,0.0,M,51.0,3.0,1.0,1.0,1.0,6.0,CAR_OTHER,OWN,4.0,806.0,NONE,4.0,RADIO_TV,6158.0,NONE,1
3933,988697,0.00,NO_CREDITS,0.00,0.0,M,31.0,1.0,1.0,0.0,1.0,2.0,SAVINGS_INSURANCE,OWN,2.0,540.0,STORES,3.0,CAR_NEW,1939.0,NONE,1
3934,337934,0.00,NO_CREDITS,0.00,0.0,M,21.0,1.0,1.0,0.0,1.0,0.0,SAVINGS_INSURANCE,RENT,2.0,124.0,STORES,3.0,RADIO_TV,2401.0,NONE,1


In [122]:
features = [
  "CHECKING_BALANCE",               # Saldo que posee el cliente en su cuenta corriente
  "PAYMENT_TERM",                   # Cantidad de días que el cvliente posee para pagar el préstamo
  "CREDIT_HISTORY",                 # Situación crediticia pasada del cliente
  "LOAN_PURPOSE",                   # Motivo del préstamo
  "LOAN_AMOUNT",                    # Monto del préstamo
  "EXISTING_SAVINGS",               # Saldo de cuenta de ahorros
  "EMPLOYMENT_DURATION",            # Cuántos años ha permanecido el cliente en su empleo
  "INSTALLMENT_PERCENT",            # Cantidad de cuotas en las que el préstamo debe ser pagado
  "SEX",                            # Sexo del cliente
  "OTHERS_ON_LOAN",                 # Denota la existencia de un garante u otro solicitante del préstamo
  "CURRENT_RESIDENCE_DURATION",     # Años que el cliente ha permanecido en su última residencia
  "PROPERTY",                       # Indica si el cliente posee alguna propiedad a su nombre
  "AGE",                            # Edad del cliente
  "INSTALLMENT_PLANS",              # Plan de financiamiento, que puede ser del banco, externo o ninguno
  "HOUSING",                        # Indica si el cliente posee una casa propia
  "EXISTING_CREDITS_COUNT",         # Número de préstamos que le han sido concedidos al cliente en el pasado
  "JOB_TYPE",                       # Tipo de empleo: 0 - desempleado, 1 - no calificado, 2 - autónomo, 3 - calificado
  "DEPENDENTS",                     # Número de personas con acceso a la cuenta
  "TELEPHONE",                      # Denota si el cliente tiene un número de teléfono registrado
  "FOREIGN_WORKER"                  # Denota si el cliente trabaja en un país fuera del banco
]
target = ['ALLOW']

In [123]:
#zero = 0.001
answer['CHECKING_BALANCE'] = pd.to_numeric(answer['CHECKING_BALANCE'], errors='coerce')
answer['EXISTING_SAVINGS'] = pd.to_numeric(answer['EXISTING_SAVINGS'], errors='coerce')
answer['FOREIGN_WORKER'].fillna(0, inplace = True)
answer[mediana] = answer[mediana].fillna(answer[mediana].median())
answer.fillna(
    value = {
        'CHECKING_BALANCE': 0
        , 'EXISTING_SAVINGS': 0 
        , 'EXISTING_CREDITS_COUNT': 0
        , 'JOB_TYPE': 0  #####
        , 'DEPENDENTS': 0
        , 'TELEPHONE': 0
        , 'FOREING_WORKER': "0" 
        , 'EMPLOYMENT_DURATION': 0 
        , 'CURRENT_RESIDENCE_DURATION': 0 
        , 'CREDIT_HISTORY': 'NO_CREDITS'
        , 'PROPERTY': 'UNKNOWN'
        , 'HOUSING': 'FREE'
        , 'INSTALLMENT_PLANS': 'NONE'
        , 'OTHERS_ON_LOAN': 'NONE'
    }
    , inplace = True
)
answer = answer.dropna()
#answer.isna().sum()

# Modelamiento

In [124]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.naive_bayes import GaussianNB 
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.ensemble import BaggingClassifier

from sklearn.preprocessing import StandardScaler

x = pd.get_dummies(full[features])
y = full[target]
answer_pred = pd.get_dummies(full[features])

rs = 23

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.45, random_state = 23)
# sclr = StandardScaler()
# x_train = sclr.fit_transform(x_train)
# x_test = sclr.fit_transform(x_test)
# answer_pred = sclr.fit_transform(answer_pred)


In [125]:
tr_mdl = DecisionTreeClassifier(max_depth = 2, random_state=rs, min_samples_split=10, min_samples_leaf = 2)
rf_mdl = RandomForestClassifier(n_estimators=500, max_depth=30, random_state=  rs)
rf_mdl1 = RandomForestClassifier(n_estimators=500, max_leaf_nodes = 16, max_depth = 30, n_jobs = -1, random_state=rs)
gb_mdl0 = GradientBoostingClassifier(random_state=12)
gb_mdl1 = GradientBoostingClassifier(n_estimators=100, random_state=rs)
nb_mdl = GaussianNB()
lg_mdl = LogisticRegression()
sv_mdl = SVC(probability=True)
sg_mdl = SGDClassifier(random_state=rs)
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators = 500,
    max_samples = 100, bootstrap = True, n_jobs = -1
)


In [126]:
mmdl = []


for mdl in (tr_mdl, rf_mdl, rf_mdl1, gb_mdl0, gb_mdl1, nb_mdl, lg_mdl, sv_mdl, sg_mdl, bag_clf):
  mdl.fit(x_train, np.ravel(y_train))
  y_pred = mdl.predict(x_test)
  #model = model.append(mdl.__class__.__name__)
  #f1 = f1.append(f1_score(y_test, y_pred))
  print(mdl.__class__.__name__, f1_score(y_test, y_pred))

DecisionTreeClassifier 0.8225140712945591
RandomForestClassifier 0.8506600660066007
RandomForestClassifier 0.8456267634018542
GradientBoostingClassifier 0.8448489863467107
GradientBoostingClassifier 0.8448489863467107
GaussianNB 0.8070973612374887
LogisticRegression 0.8271298593879238


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


SVC 0.8168902920284137
SGDClassifier 0.692787177203918
BaggingClassifier 0.842366570128258


In [128]:
answer["ALLOW"] = pd.DataFrame(rf_mdl.predict(answer_pred))
# answer.info()

In [129]:
answer.to_csv("out.csv", index = False)